# IMPORTING LIBRARIES

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options
option = webdriver.ChromeOptions()
option.add_argument('headless')
from IPython.display import clear_output
import cv2
from PIL import Image
import numpy as np

# Opening chrome driver (headless mode)

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\ND.COM\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


# Path for saving temp screenshot

In [3]:
img_path="temp_img.jpg"

# Function- Screenshot

In [4]:
def take_ss(address):
    global img_path,driver
    url=f"https://www.google.com/maps/place/{address}"
    driver.get(url)
    time.sleep(2)
    driver.save_screenshot(img_path)

# Function- Image processing 

In [5]:
def image_processing():
    global img_path
    img=cv2.imread(img_path)
    img=img[:,408:]
    
    #img to hsv
    img_hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # lower mask (0-10)
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

    # join my masks
    mask = mask0+mask1

    # set my output img to zero everywhere except my mask
    output_img = img.copy()
    output_img[np.where(mask==0)] = 0

    # or your HSV image, which I *believe* is what you want
    output_hsv = img_hsv.copy()
    output_hsv[np.where(mask==0)] = 0
    
    h, s, v1 = cv2.split(output_hsv)
    
    contours, hierarchy = cv2.findContours(v1,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    null=True
    new_contours=[]
    img_contour=img.copy()
    img_fill_contour=img.copy()

    for con in contours:
        area = cv2.contourArea(con)
        x,y,w,h = cv2.boundingRect(con)
        if h>20 and w>30:
            null=False
            new_contours.append(con)
            cv2.drawContours(img_contour, con, -1, (40, 245, 255),3) 

    return null

# Area Classification (Yes/No)

In [ ]:
#MAIN
while(True):
    address=input("Note:\n1-Make sure you have connected to a stable internet connection.\n2-Spelling should be correct\n3-Write area along with city separated by ',' for accurate classification e.g 'saddar, karachi' \n\nEnter area/address: ")
    print("\nProcessing..")
    take_ss(address)
    null=image_processing()

    if null:
        clear_output(wait=True)
        print(f"****************** RESULT ******************\n\nArea: {address.upper()}\nIs it an area according to Google maps: NO")
    else:
        clear_output(wait=True)
        print(f"****************** RESULT ******************\n\nArea: {address.upper()}\nIs it an area according to Google maps: YES")
    print("\n********************************************")
    print("\n\nDo you want to check again?\n\nPress C to continue or anyother key to exit: ")
    check=input("").lower()
    if check=="c":
        clear_output(wait=True)
        pass
    else:
        clear_output(wait=True)
        print("Thank you for trying!")
        break
    
    